In [ ]:
import os
import re
import cv2 as cv
import sys
import csv
import time
import math
import pickle
import string
import pyprind
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [6]:
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras import backend as K
from keras_adabound import AdaBound

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, LSTM, Input, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, TensorBoard
import itertools
import xgboost as xgb

Using TensorFlow backend.


# Preprocessing

In [8]:
test_images = pd.read_pickle('./test_images.pkl')
train_images = pd.read_pickle('./train_images.pkl')
train_labels = pd.read_csv('./train_labels.csv')

# Preprocessing
x = train_images
y = train_labels['Category'].values
x = x.reshape(-1, 64, 64)
y = y.reshape(-1, 1) 
x = np.int64(x)
y = np.int64(y)


def preprocess(image):
    connectivity = 4
    img = image.astype(np.uint8)
    _,thresh = cv.threshold(img,254,255,cv.THRESH_BINARY)

    
    img_blurred = cv.blur(img,(3,3))

    _,thresh_blurred = cv.threshold(img_blurred,205,255,cv.THRESH_BINARY)    

    output = cv.connectedComponentsWithStats(thresh_blurred, connectivity, cv.CV_32S)
    n_components = output[0] - 1
    stats = output[2][1:]
    
    # Too many high intensity parts, go without blur
    if n_components >= 4:
        output = cv.connectedComponentsWithStats(thresh, connectivity, cv.CV_32S)
        n_components = output[0] - 1
        stats = output[2][1:]
    
     
    largest_component = np.array([])
    largest_area = 0
    largest_length = 0
    largest_height = 0
    for i in range(n_components):
        
        left = stats[i][0]
        top = stats[i][1]
        length = stats[i][2]
        height = stats[i][3]
        if max(length,height) >=32:
            continue
        area = max(length,height) ** 2 # for square bounding box
        
        if(area > largest_area):
            largest_area = area
            largest_length = length
            largest_height = height
            largest_component = thresh[top:height+top, left:left+length]

            
    result = np.zeros([64,64],dtype=np.uint8)
    result[0:largest_height,0:largest_length] = result[0:largest_height,0:largest_length] + largest_component    
    result = result[:32,:32]
    return result

x1 = np.zeros([x.shape[0],32,32],dtype=np.uint8)
test1 = np.zeros([x.shape[0],32,32],dtype=np.uint8)
for i in range(x.shape[0]):
    x1[i] = preprocess(x[i])

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(x1, y, test_size=0.2)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [6]:
np.array(train_images_p).shape

In [11]:
x_train = X_train.reshape(X_train.shape[0], -1)
x_test = X_test.reshape(X_test.shape[0], -1)

x_train = x_train/255.0
x_test = x_test/255.0



## Random Forest

In [28]:
seed = 101
rfc = RandomForestClassifier(n_jobs=1, n_estimators=500, max_features='auto',random_state=seed)

In [29]:
params_rfc = {"n_estimators": [500, 1000],
              "max_features": [500, 1000, 'auto'],
              "criterion": ['gini', 'entropy']}
rfc_cv = GridSearchCV(rfc, param_grid = params_rfc, cv=2, verbose = 1, n_jobs = -1)

In [30]:
rfc_cv.fit(x_train, Y_train)
cv_tabular(rfc_cv.cv_results_) 

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done  12 out of  24 | elapsed: 21.1min remaining: 21.1min
[Parallel(n_jobs=-1)]: Done  15 out of  24 | elapsed: 34.8min remaining: 20.9min
[Parallel(n_jobs=-1)]: Done  18 out of  24 | elapsed: 35.0min remaining: 11.7min
[Parallel(n_jobs=-1)]: Done  21 out of  24 | elapsed: 36.6min remaining:  5.2min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 50.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 50.7min finished


,criterion,max_features,n_estimators,mean_test_score,std_test_score,mean_fit_time
2,gini,1000,500,0.859406,0.000191,1015.995549
3,gini,1000,1000,0.859437,0.000277,2014.391321
9,entropy,1000,1000,0.861781,0.003497,1762.063940
8,entropy,1000,500,0.861844,0.004122,1021.501806
1,gini,500,1000,0.873812,0.001282,1047.722257
0,gini,500,500,0.874031,0.001125,516.872876
6,entropy,500,500,0.875531,0.001250,544.230008
7,entropy,500,1000,0.875687,0.000844,1079.722607
10,entropy,auto,500,0.888437,0.000403,51.912349
11,entropy,auto,1000,0.888719,0.000059,89.797717


In [36]:
rfc_cv.score(x_test, Y_test)

0.898

## k-NN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(x_train,Y_train)
knn.score(x_test, Y_test)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs',C=1.0,verbose=1,n_jobs=-1,max_iter=800)
lr.fit(x_train,Y_train)
print('finish training')
result = lr.predict(x_test).transpose().astype(int).flatten()
print('finish validation')
total = len(Y_test)
correct = (result==Y_test).sum()
accuracy = correct / total
print('accuracy = ' + str(accuracy))